In [ ]:

import ipywidgets as widgets
from IPython.display import display
from datetime import date

layout_input = widgets.Layout(width='100%')
layout_button = widgets.Layout(width='100%', height='40px')
layout_half = widgets.Layout(width='48%')
layout_progress = widgets.Layout(width='100%', height='20px')

todo_items = []

def get_bar_style(progress):
    if progress == 0:
        return '#FFFFFF'
    gray_value = int(255 - (progress * 2.55))
    return f'rgb({gray_value},{gray_value},{gray_value})'

overall_progress_label = widgets.Label(value='최종 진행률', style={'font_weight': 'bold'})
overall_progress = widgets.FloatProgress(min=0, max=100, value=0, layout=widgets.Layout(width='90%', height='20px'))
overall_percent_label = widgets.Label(value='(0%)')

def update_overall_progress():
    if not todo_items:
        overall_progress.value = 0
        overall_percent_label.value = '(0%)'
        overall_progress.bar_style = ''
    else:
        avg = sum(item['progress'] for item in todo_items) / len(todo_items)
        overall_progress.value = avg
        overall_percent_label.value = f'({int(avg)}%)'
        overall_progress.style = {'bar_color': get_bar_style(avg)}

def create_task_ui(title, progress, due_date, priority, index):
    title_label = widgets.HTML(value=f"<b># {index + 1}. {title}</b>")
    progress_bar = widgets.FloatProgress(value=progress, min=0, max=100, layout=widgets.Layout(width='80%', height='20px'))
    progress_bar.style = {'bar_color': get_bar_style(progress)}
    slider = widgets.IntSlider(value=progress, min=0, max=100, step=5, layout=widgets.Layout(width='80%', height='20px'))

    def on_slider_change(change):
        progress_bar.value = change['new']
        todo_items[index]['progress'] = change['new']
        progress_bar.style = {'bar_color': get_bar_style(change['new'])}
        update_overall_progress()

    slider.observe(on_slider_change, names='value')

    due_label = widgets.Label(f"{due_date}")
    priority_label = widgets.Label(priority)

    complete_btn = widgets.Button(description='완료✅', layout=layout_half)
    delete_btn = widgets.Button(description='삭제❌', layout=layout_half)
    button_box = widgets.HBox([complete_btn, delete_btn], layout=layout_input)

    def mark_complete(b):
        slider.value = 100

    def delete_item(b):
        todo_items.pop(index)
        refresh_task_list()

    complete_btn.on_click(mark_complete)
    delete_btn.on_click(delete_item)

    task_ui = widgets.VBox([
        title_label,
        progress_bar,
        widgets.HBox([widgets.Label("마감일"), due_label, widgets.Label("중요도"), priority_label]),
        button_box
    ], layout=widgets.Layout(border='1px solid #ccc', padding='10px', margin='10px 0'))

    return task_ui

task_list = widgets.VBox()

def refresh_task_list():
    task_list.children = [create_task_ui(**item, index=i) for i, item in enumerate(todo_items)]
    update_overall_progress()

title_input = widgets.Text(placeholder='할 일 입력', layout=layout_input)
progress_input = widgets.IntSlider(value=0, min=0, max=100, step=5, layout=layout_input)
due_input = widgets.DatePicker(layout=layout_half)
priority_input = widgets.Dropdown(options=['낮음', '보통', '높음'], value='보통', layout=layout_half)
add_button = widgets.Button(description='추가', layout=layout_input)

save_button = widgets.Button(description='저장', button_style='info', layout=layout_half)
load_button = widgets.Button(description='불러오기', button_style='warning', layout=layout_half)

def add_task(b):
    if not title_input.value.strip():
        return
    todo_items.append({
        'title': title_input.value.strip(),
        'progress': progress_input.value,
        'due_date': due_input.value or date.today(),
        'priority': priority_input.value
    })
    title_input.value = ''
    progress_input.value = 0
    due_input.value = None
    priority_input.value = '보통'
    refresh_task_list()

def save_tasks(b):
    with open('tasks.json', 'w') as f:
        import json
        json.dump(todo_items, f, default=str)

def load_tasks(b):
    try:
        with open('tasks.json', 'r') as f:
            import json
            from datetime import datetime
            raw = json.load(f)
            todo_items.clear()
            for item in raw:
                todo_items.append({
                    'title': item['title'],
                    'progress': item['progress'],
                    'due_date': datetime.strptime(item['due_date'], '%Y-%m-%d').date(),
                    'priority': item['priority']
                })
            refresh_task_list()
    except:
        pass

add_button.on_click(add_task)
save_button.on_click(save_tasks)
load_button.on_click(load_tasks)

header = widgets.HTML("<h3>📂 DAVID mk42</h3>")
input_widgets = widgets.VBox([
    widgets.HBox([widgets.Label("할 일", layout=layout_half), title_input]),
    widgets.HBox([widgets.Label("진행률", layout=layout_half), progress_input]),
    widgets.HBox([widgets.Label("마감일", layout=layout_half), due_input,
                  widgets.Label("중요도", layout=layout_half), priority_input]),
    add_button,
    widgets.HBox([save_button, load_button]),
    widgets.Label("최종 진행률", layout=widgets.Layout(justify_content='center', width='100%', text_align='center'), style={'font_weight': 'bold'}),
    widgets.HBox([widgets.Label('진행률'), overall_progress, overall_percent_label])
])

display(widgets.VBox([header, input_widgets, task_list]))
refresh_task_list()